In [ ]:
%pip install llama-index-readers-file --quiet
%pip install llama-index-embeddings-openai --quiet
%pip install llama-index-agent-openai --quiet
%pip install llama-index-llms-openai --quiet

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["OPENAI_API_KEY"] = ''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Building Agentic RAG with Llamaindex/metagpt.pdf"]).load_data()

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [ ]:
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores import MetadataFilters

def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o", temperature=0)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "agent roles in MetaGPT", "page_numbers": []}
=== Function Output ===
In MetaGPT, there are five specified agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and expertise tailored to different aspects of the software development process within the framework. The roles are designed to work collaboratively, with each agent contributing specialized outputs aligned with their designated tasks and skills.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "how agents communicate with each other in MetaGPT", "page_numbers": []}
=== Function Output ===
Agents in MetaGPT communicate with each other by utilizing a shared message pool where they publish structured messages and subscribe to relevant message

In [ ]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 9
file_name: metagpt.pdf
file_path: /content/drive/MyDrive/Colab Notebooks/Building Agentic RAG with Llamaindex/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-02
last_modified_date: 2024-05-31

Preprint
Table 2: Comparison of capabilities for MetaGPT and other approaches. ‘!’ indicates the
presence of a specific feature in the corresponding framework, ‘ %’ its absence.
Framework Capabiliy AutoGPT LangChain AgentVerse ChatDev MetaGPT
PRD generation % % % % !
Tenical design genenration % % % % !
API interface generation % % % % !
Code generation ! ! ! ! !
Precompilation execution % % % % !
Role-based task management % % % ! !
Code review % % ! ! !
Table 3: Ablation study on roles. ‘#’ denotes ‘The number of’, ‘Product’ denotes ‘Product man-
ager’, and ‘Project’ denotes ‘Project manager’. ‘ !’ indicates the addition of a specific role. ‘Revi-
sions’ refers to ‘Human Revision Cost’.
Engineer Product Architect Project #Agents #Lines Expense Rev

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "evaluation datasets used in MetaGPT", "page_numbers": []}
=== Function Output ===
The evaluation datasets used in MetaGPT are the SoftwareDev dataset and the MBPP and HumanEval benchmarks.
=== LLM Response ===
### Evaluation Datasets Used in MetaGPT

MetaGPT utilizes the following evaluation datasets:

1. **SoftwareDev Dataset**: This dataset is specifically designed to evaluate the performance of MetaGPT in the context of software development tasks. It includes various scenarios and tasks that agents might encounter in real-world software projects.

2. **MBPP (Mostly Basic Python Problems)**: This benchmark consists of a collection of Python programming problems that range from basic to intermediate levels. It is used to assess the coding and problem-solving capabilities of the agents.

3. **HumanEval**: This benchmark includes a set of

In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "results over evaluation datasets in MetaGPT", "page_numbers": []}
=== Function Output ===
MetaGPT achieves an average score of 3.9, surpassing ChatDev's score of 2.1. When compared to general intelligent algorithms like AutoGPT, which score 1.0, MetaGPT demonstrates superior performance. Additionally, MetaGPT outperforms preceding approaches in both HumanEval and MBPP benchmarks, achieving 85.9% and 87.7% respectively.
=== LLM Response ===
### Results Over Evaluation Datasets in MetaGPT

MetaGPT has demonstrated impressive performance across various evaluation datasets:

1. **General Performance**:
   - **MetaGPT**: Achieves an average score of 3.9.
   - **ChatDev**: Scores 2.1.
   - **AutoGPT**: Scores 1.0.

   MetaGPT significantly outperforms both ChatDev and AutoGPT, showcasing its superior capabilities.

2. **HumanEval Bench

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "agent roles in MetaGPT", "page_numbers": []}
=== Function Output ===
In MetaGPT, there are five specified agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and expertise tailored to their designated tasks within the software company framework. These roles are crucial for the effective functioning of the multi-agent system and contribute to the overall success of the code generation process.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "how agents communicate with each other in MetaGPT", "page_numbers": []}
=== Function Output ===
Agents in MetaGPT communicate with each other by utilizing a shared message pool where they publish structured messages and subscribe to relevant messages based o

In [ ]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task cd189117-2341-4b46-91a2-b29ef4f02695: 1
In MetaGPT, there are five specified agent roles: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and expertise tailored to their designated tasks within the software company framework. These roles are crucial for the effective functioning of the multi-agent system and contribute to the overall success of the code generation process.


In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task cd189117-2341-4b46-91a2-b29ef4f02695: 1


TaskStep(task_id='cd189117-2341-4b46-91a2-b29ef4f02695', step_id='678f9d70-8b57-451e-b195-ac836e39effc', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: vector_tool with args: {"query": "how agents share information in MetaGPT", "page_numbers": []}
=== Function Output ===
Agents in MetaGPT share information by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, eliminating the need for one-to-one communication. Additionally, agents can access messages from other entities transparently, enhancing communication efficiency. To manage information overload, agents can use a subscription mechanism to extract only relevant information based on their role profiles.


In [ ]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")


Num upcoming steps for task cd189117-2341-4b46-91a2-b29ef4f02695: 1


In [ ]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, agents share information through a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant ones based on their profiles. This system facilitates direct message exchange among agents, eliminating the need for one-to-one communication. Agents can transparently access messages from other entities, enhancing communication efficiency. To manage information overload, agents use a subscription mechanism to extract only the information pertinent to their roles.
True


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

In MetaGPT, agents share information through a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant ones based on their profiles. This system facilitates direct message exchange among agents, eliminating the need for one-to-one communication. Agents can transparently access messages from other entities, enhancing communication efficiency. To manage information overload, agents use a subscription mechanism to extract only the information pertinent to their roles.
